In [2]:
import spatial_embedings
from jax import random, numpy as jnp
import extra_initializers
from flax.linen import initializers
import matplotlib.pyplot as plt
import models

In [4]:
initializers.ones_init()

<function jax.nn.initializers.ones(key: 'KeyArray', shape: 'core.Shape', dtype: 'DTypeLikeInexact' = <class 'jax.numpy.float64'>) -> 'Array'>

In [5]:
error = jnp.array([1,2,3]) #(3,)
kernel = jnp.array([[1,2],[1,3],[4,5]]) # (3,2)
error.shape

(1, 3)

In [6]:
jnp.dot(error, kernel)

Array([[15, 23]], dtype=int32)

(3, 1, 2)

In [5]:
w = 4
h = 3
n_cells = 10
seed = 1
key = random.key(seed)

In [6]:
grid = spatial_embedings.twod_grid(w, h)
grid

Array([[0.        , 0.        ],
       [0.25      , 0.        ],
       [0.5       , 0.        ],
       [0.75      , 0.        ],
       [0.        , 0.33333334],
       [0.25      , 0.33333334],
       [0.5       , 0.33333334],
       [0.75      , 0.33333334],
       [0.        , 0.6666667 ],
       [0.25      , 0.6666667 ],
       [0.5       , 0.6666667 ],
       [0.75      , 0.6666667 ]], dtype=float32)

In [8]:
cells_indices = spatial_embedings.cell_to_twod_grid(w=w, h=h, n_cells=n_cells, key=key)
cells_indices

Array([[1, 2],
       [1, 3],
       [2, 2],
       [1, 1],
       [0, 2],
       [2, 0],
       [0, 0],
       [1, 0],
       [0, 3],
       [2, 1]], dtype=int32)

In [ ]:
# convert 2d index coding to 1d
row_indices = cells_indices[:, 0]
col_indices = cells_indices[:, 1]
oned_indices = row_indices * w + col_indices

In [ ]:
# get selected cells in grid
selected_cells = grid[oned_indices, :]
# Get x and y positions
x = selected_cells[:, 0]
y = selected_cells[:, 1]
cells_indices

In [ ]:
sigma = 0.012

M =spatial_embedings.twodMatrix(Pre_x=x, Pre_y=y, Post_x=x, Post_y=y, sigma=sigma, key=key)


In [ ]:
plt.imshow(M)

In [ ]:
degree = jnp.mean(M) * n_cells
print(degree)

In [ ]:
model = models.LSSN(n_ALIF=50, n_LIF=50, local_connectivity_key=seed, cell_loc_key=seed, sigma = 0.012)

In [ ]:
def get_initial_params(rng, model, input_shape):
  """Returns randomly initialized parameters, eligibility parameters and connectivity mask."""
  dummy_x = jnp.ones(input_shape)
  variables = model.init(rng, dummy_x)
  return variables['params'], variables['eligibility params'], variables[ 'spatial embbeding']

In [ ]:
rng = random.key(seed)
input_shape = (40, 10)

In [ ]:
params, eligibility_variables, spatial_embedding = get_initial_params(key, model, input_shape)

In [ ]:
M = spatial_embedding["ALIFCell_0"]['M']

In [ ]:
plt.imshow(M)

In [ ]:
jnp.mean(M) * n_cells

In [ ]:
M.shape

In [ ]:
spatial_embedding["ALIFCell_0"]["cells_loc"]